In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data_path = '/content/drive/MyDrive/ACDC/database/training'

In [3]:
import os
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import nibabel as nib
import cv2

In [4]:
list_data = os.listdir(data_path)

In [5]:
len(list_data)

101

In [6]:
samples = list_data
samples

['patient015',
 'patient016',
 'patient013',
 'patient014',
 'patient011',
 'patient012',
 'patient009',
 'patient010',
 'patient007',
 'patient008',
 'patient005',
 'patient006',
 'patient003',
 'patient004',
 'patient043',
 'patient044',
 'patient045',
 'patient046',
 'patient047',
 'patient048',
 'patient049',
 'patient050',
 'patient039',
 'patient040',
 'patient037',
 'patient038',
 'patient035',
 'patient036',
 'patient033',
 'patient034',
 'patient031',
 'patient032',
 'patient063',
 'patient064',
 'patient065',
 'patient066',
 'patient067',
 'patient068',
 'patient069',
 'patient070',
 'patient071',
 'patient072',
 'patient079',
 'patient080',
 'patient055',
 'patient056',
 'patient057',
 'patient058',
 'patient059',
 'patient060',
 'patient075',
 'patient076',
 'patient077',
 'patient078',
 'patient081',
 'patient082',
 'patient097',
 'patient098',
 'patient099',
 'patient100',
 'patient089',
 'patient090',
 'patient087',
 'patient088',
 'patient091',
 'patient092',
 'patient0

In [8]:
new_data = {
    'x':[],
    'y':[]
}
for sample in samples:
  try:
    folder_path = os.path.join(data_path, sample)
    list_file = os.listdir(folder_path)
    print(list_file)
    # load classification groundtruth
    with open(os.path.join(folder_path, 'Info.cfg'), 'r') as f:
      value = f.read().split('\n')
      value = {v.split(':')[0]: v.split(':')[1].strip() for v in value if ':' in v}
      group = value['Group']

    file_to_be_loaded = [l for l in list_file if 'gt' in l][0]
    print(file_to_be_loaded)
    # load classification input
    nii_img  = nib.load(os.path.join(folder_path, file_to_be_loaded))
    nii_data = nii_img.get_fdata()
    if nii_data.shape[-1]<18:
      nii_data_temp = np.zeros(nii_data.shape[:2]+(18,))
      nii_data_temp[:,:,:nii_data.shape[-1]] = nii_data
      nii_data = nii_data_temp

    new_nii_data = cv2.resize(nii_data, (256,256))
    print(new_nii_data.shape)

    new_data['x'].append(new_nii_data)
    new_data['y'].append(group)
  except:
    print('ignore')



['MANDATORY_CITATION.md', 'patient015_4d.nii.gz', 'Info.cfg', 'patient015_frame01.nii.gz', 'patient015_frame10_gt.nii.gz', 'patient015_frame01_gt.nii.gz', 'patient015_frame10.nii.gz']
patient015_frame10_gt.nii.gz
(256, 256, 18)
['patient016_4d.nii.gz', 'patient016_frame01_gt.nii.gz', 'patient016_frame01.nii.gz', 'patient016_frame12_gt.nii.gz', 'MANDATORY_CITATION.md', 'patient016_frame12.nii.gz', 'Info.cfg']
patient016_frame01_gt.nii.gz
(256, 256, 18)
['MANDATORY_CITATION.md', 'Info.cfg', 'patient013_4d.nii.gz', 'patient013_frame14_gt.nii.gz', 'patient013_frame01_gt.nii.gz', 'patient013_frame01.nii.gz', 'patient013_frame14.nii.gz']
patient013_frame14_gt.nii.gz
(256, 256, 18)
['patient014_4d.nii.gz', 'MANDATORY_CITATION.md', 'Info.cfg', 'patient014_frame01_gt.nii.gz', 'patient014_frame13_gt.nii.gz', 'patient014_frame01.nii.gz', 'patient014_frame13.nii.gz']
patient014_frame01_gt.nii.gz
(256, 256, 18)
['patient011_4d.nii.gz', 'Info.cfg', 'MANDATORY_CITATION.md', 'patient011_frame08.nii.gz

In [9]:
new_data

{'x': [array([[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         ...,
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
    

In [11]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Assuming 'x' is your data and 'y' is your labels
length = set([l.shape[-1] for l in new_data['x']])
print(length)
x = np.array(new_data['x']) # Replace this with your real data
y = new_data['y']  # Replace this with your real labels

# Convert labels to integer encoding
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
print(len(x_train), len(y_train))

# Create CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 18)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(set(y)), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


{18}
90 90
Epoch 1/100
3/3 [==============================] - 3s 296ms/step - loss: 7.5448 - accuracy: 0.2840 - val_loss: 5.1727 - val_accuracy: 0.1111
Epoch 2/100
3/3 [==============================] - 0s 147ms/step - loss: 2.9717 - accuracy: 0.3827 - val_loss: 1.7040 - val_accuracy: 0.2222
Epoch 3/100
3/3 [==============================] - 0s 145ms/step - loss: 1.2924 - accuracy: 0.6049 - val_loss: 1.8013 - val_accuracy: 0.2222
Epoch 4/100
3/3 [==============================] - 0s 143ms/step - loss: 0.9694 - accuracy: 0.6667 - val_loss: 2.0209 - val_accuracy: 0.2222
Epoch 5/100
3/3 [==============================] - 0s 138ms/step - loss: 0.5345 - accuracy: 0.8272 - val_loss: 2.4407 - val_accuracy: 0.3333
Epoch 6/100
3/3 [==============================] - 0s 146ms/step - loss: 0.2843 - accuracy: 0.9012 - val_loss: 1.8168 - val_accuracy: 0.4444
Epoch 7/100
3/3 [==============================] - 0s 142ms/step - loss: 0.1473 - accuracy: 0.9753 - val_loss: 1.7572 - val_accuracy: 0.5556
Ep